<a href="https://colab.research.google.com/github/AlexandreXavierMartins/Sim/blob/main/simpy_aula_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install simpy

Sistema Aberto

In [8]:
import simpy
import numpy as np

np.random.seed(0)

ocp = 0
tempo_total_fila = 0
cont_na_fila = 0

def carros(env):
  cont_carros = 0
  while True:
    chegada = np.random.exponential(30)
    yield env.timeout(chegada)
    cont_carros += 1
    print('%.1f Chegada do carro %d' % (env.now, cont_carros))

env = simpy.Environment()
env.process(carros(env))

env.run(until=1000)

23.9 Chegada do carro 1
61.6 Chegada do carro 2
89.3 Chegada do carro 3
112.9 Chegada do carro 4
129.4 Chegada do carro 5
160.5 Chegada do carro 6
177.8 Chegada do carro 7
244.5 Chegada do carro 8
344.0 Chegada do carro 9
358.5 Chegada do carro 10
405.5 Chegada do carro 11
428.1 Chegada do carro 12
453.3 Chegada do carro 13
531.2 Chegada do carro 14
533.5 Chegada do carro 15
536.2 Chegada do carro 16
536.8 Chegada do carro 17
590.4 Chegada do carro 18
635.6 Chegada do carro 19
696.8 Chegada do carro 20
812.2 Chegada do carro 21
860.3 Chegada do carro 22
878.9 Chegada do carro 23
924.4 Chegada do carro 24
928.2 Chegada do carro 25
958.8 Chegada do carro 26
963.5 Chegada do carro 27


Exercício 1: Crie o processo de atendimento do carro no posto de pedágio. O tempo de atendimento é EXPO(20)
a) Colete a ocupação do atendente
b) O tempo médio de fila
c) O tempo máximo na fila
d) O número médio da fila
e) Verifique o tempo ideal de simulação que mostra que os dados não se alteram muito

Exercício 2: Considere uma lavanderia com 4 lavadoras, 3 secadoras e 5 cestos de roupas. Quando um cliente chega (taxa Exponential(3)), ele coloca as roupas em uma máquina de lavar (ou aguarda em fila). A lavagem consome 20 minutos (constante). Ao terminar a lavagem, o cliente retira as roupas da máquina e coloca em um cesto e leva o cesto com suas roupas até a secadora, num processo que leva de 1 a 4 minutos distribuídos uniformemente. O cliente então descarrega as roupas do cesto diretamente para a secadora, espera a secagem e vai embora. Esse processo leva entre 9 e 12 minutos, uniformemente distribuídos. Construa um modelo de simulação que represente este processo. Simule por 10 horas e verifique o gargalo do sistema.

Exercício 3: Devido ao alto número de caminhões na fila do sistema da carregadeira o gerente decidiu fazer a simulação do sistema. Sendo assim o seu funcionamento pode ser descrito da seguinte forma: os caminhões são carregados pela FX215, gastando em média Normal(200, 15) segundos, depois seguem para o local onde está o britador com um tempo de Normal(90, 5) segundos, chegando lá eles descarregam com um tempo de Normal(50, 7) segundos e voltam para o local onde está a carregadeira com um tempo de Normal(70, 6) segundos. Simule o sistema.